Project guide: https://www.dataquest.io/m/240/guided-project%3A-predicting-house-sale-prices

Solution by DataQuest: https://github.com/dataquestio/solutions/blob/master/Mission240Solutions.ipynb

This project uses "a dataset on sold houses in Ames, Iowa. Each row in the dataset describes the properties of a single house as well as the amount it was sold for." ([source](https://www.dataquest.io/m/235/the-linear-regression-model/2/introduction-to-the-data)). The dataset can be download [here](https://ww2.amstat.org/publications/jse/v19n3/decock/AmesHousing.txt). Each column in the dataset is described in [DataDocumentation.txt](https://ww2.amstat.org/publications/jse/v19n3/decock/DataDocumentation.txt).

I will predict `SalePrice` of each house based on its other attributes. Linear regression will be done with [ordinary least squares](https://en.wikipedia.org/wiki/Ordinary_least_squares).

# 1. Data overview

In [1]:
from IPython.display import display
from pandas.api.types import CategoricalDtype
from pprint import pprint
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.feature_extraction import FeatureHasher
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.svm import SVR
from time import time

import numpy as np
import pandas as pd
import re

# Load data
df = pd.read_csv("AmesHousing.txt", delimiter="\t")

# Ensure no rows or columns are ommited from display
pd.options.display.max_rows = df.shape[0]
pd.options.display.max_columns = df.shape[1]

print("Data frame dimension")
print("Number of rows:", df.shape[0])
print("Number of columns:", df.shape[1])

# First 5 rows
display(df.head(5))

Data frame dimension
Number of rows: 2930
Number of columns: 82


,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating,Heating QC,Central Air,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,5,1960,1960,Hip,CompShg,BrkFace,Plywood,Stone,112.0,TA,TA,CBlock,TA,Gd,Gd,BLQ,639.0,Unf,0.0,441.0,1080.0,GasA,Fa,Y,SBrkr,1656,0,0,1656,1.0,0.0,1,0,3,1,TA,7,Typ,2,Gd,Attchd,1960.0,Fin,2.0,528.0,TA,TA,P,210,62,0,0,0,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,GasA,TA,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1961.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,GasA,TA,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,1958.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,7,5,1968,1968,Hip,CompShg,BrkFace,BrkFace,None,0.0,Gd,TA,CBlock,TA,TA,No,ALQ,1065.0,Unf,0.0,1045.0,2110.0,GasA,Ex,Y,SBrkr,2110,0,0,2110,1.0,0.0,2,1,3,1,Ex,8,Typ,2,TA,Attchd,1968.0,Fin,2.0,522.0,TA,TA,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,GasA,Gd,Y,SBrkr,928,701,0,1629,0.0,0.0,2,1,3,1,TA,6,Typ,1,TA,Attchd,1997.0,Fin,2.0,482.0,TA,TA,Y,212,34,0,0,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


In [2]:
# Show numeric data only
df.select_dtypes(include=np.number).head()

,Order,PID,MS SubClass,Lot Frontage,Lot Area,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Mas Vnr Area,BsmtFin SF 1,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,TotRms AbvGrd,Fireplaces,Garage Yr Blt,Garage Cars,Garage Area,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold,SalePrice
0,1,526301100,20,141.0,31770,6,5,1960,1960,112.0,639.0,0.0,441.0,1080.0,1656,0,0,1656,1.0,0.0,1,0,3,1,7,2,1960.0,2.0,528.0,210,62,0,0,0,0,0,5,2010,215000
1,2,526350040,20,80.0,11622,5,6,1961,1961,0.0,468.0,144.0,270.0,882.0,896,0,0,896,0.0,0.0,1,0,2,1,5,0,1961.0,1.0,730.0,140,0,0,0,120,0,0,6,2010,105000
2,3,526351010,20,81.0,14267,6,6,1958,1958,108.0,923.0,0.0,406.0,1329.0,1329,0,0,1329,0.0,0.0,1,1,3,1,6,0,1958.0,1.0,312.0,393,36,0,0,0,0,12500,6,2010,172000
3,4,526353030,20,93.0,11160,7,5,1968,1968,0.0,1065.0,0.0,1045.0,2110.0,2110,0,0,2110,1.0,0.0,2,1,3,1,8,2,1968.0,2.0,522.0,0,0,0,0,0,0,0,4,2010,244000
4,5,527105010,60,74.0,13830,5,5,1997,1998,0.0,791.0,0.0,137.0,928.0,928,701,0,1629,0.0,0.0,2,1,3,1,6,1,1997.0,2.0,482.0,212,34,0,0,0,0,0,3,2010,189900


# 2. Feature engineering - part 1

## 2.1. Format conversion

Numerical and text data will be convered to [pandas categorical format](https://pandas.pydata.org/pandas-docs/stable/categorical.html).

Features with ordered and unordered categories will be converted separately.

**Note**: The ordered and unordered categorical data will not be analysed separately because I currently do not know methods suitable to each type.

### 2.1.1. Numeric to categorical (unordered)

In [3]:
ms_subclass_map = {20: "1-STORY 1946 & NEWER ALL STYLES", 
                    30: "1-STORY 1945 & OLDER", 
                    40: "1-STORY W/FINISHED ATTIC ALL AGES", 
                    45: "1-1/2 STORY - UNFINISHED ALL AGES", 
                    50: "1-1/2 STORY FINISHED ALL AGES", 
                    60: "2-STORY 1946 & NEWER", 
                    70: "2-STORY 1945 & OLDER", 
                    75: "2-1/2 STORY ALL AGES", 
                    80: "SPLIT OR MULTI-LEVEL", 
                    85: "SPLIT FOYER", 
                    90: "DUPLEX - ALL STYLES AND AGES", 
                    120: "1-STORY PUD (Planned Unit Development) - 1946 & NEWER", 
                    150: "1-1/2 STORY PUD - ALL AGES", 
                    160: "2-STORY PUD - 1946 & NEWER", 
                    180: "PUD - MULTILEVEL - INCL SPLIT LEV/FOYER", 
                    190: "2 FAMILY CONVERSION - ALL STYLES AND AGES"}
    
df["MS SubClass"] = df["MS SubClass"]\
                        .map(ms_subclass_map)\
                        .astype("category")

### 2.1.2. Numeric to categorical (ordered)

In [4]:
overall_cats = ['Very Poor',
                 'Poor',
                 'Fair',
                 'Below Average',
                 'Average',
                 'Above Average',
                 'Good',
                 'Very Good',
                 'Excellent',
                 'Very Excellent']

overal_vals = sorted(df["Overall Qual"].unique())

overall_map = {val: overall_cats[val - 1] for val in overal_vals}

# ensure categories are given orders (source https://bit.ly/2JqhcKd)
ord_num_cat_type = CategoricalDtype(categories=overall_cats, ordered=True)

ord_num_cols = ["Overall Cond", "Overall Qual"]
df[ord_num_cols] = df[ord_num_cols].applymap(lambda x: overall_map[x])
for col in ord_num_cols:
    df[col] = df[col].astype(ord_num_cat_type)

### 2.1.3. Text to categorical (unordered)

In [5]:
unord_txt_cols = ["MS SubClass", "MS Zoning", "Street", "Alley", 
                 "Land Contour", "Lot Config", "Neighborhood", 
                 "Condition 1", "Condition 2", "Bldg Type", 
                 "House Style", "Roof Style", "Roof Matl", 
                 "Exterior 1st", "Exterior 2nd", "Mas Vnr Type", 
                 "Foundation", "Heating", "Central Air", 
                 "Garage Type", "Misc Feature", "Sale Type", 
                 "Sale Condition"]

# source https://stackoverflow.com/a/28910914
for col in unord_txt_cols:
    df[col] = df[col].astype("category")

### 2.1.4. Text to categorical (ordered)

First, each feature's categories will be extracted from [DataDocumentation.txt](https://ww2.amstat.org/publications/jse/v19n3/decock/DataDocumentation.txt).

In [6]:
ordinal = False

ord_txt_cols = {}

with open("DataDocumentation.txt", "rb") as f:
    for row in f.read().splitlines():
        try:
            row = row.decode("utf-8")
            if "(Ordinal)" in row:
                ordinal = True
                key = re.sub("[\ |\t]\(Ordinal.*", "", row)
                if key not in ord_num_cols:
                    ord_txt_cols[key] = []
            elif ("(Nominal)" in row) or ("Discrete" in row) \
                        or ("(Continuous)" in row):
                ordinal = False
            else:
                if row.split() == []:
                    pass
                elif ordinal:
                    strip_first_spaces = re.sub("^\ *", "", row)
                    strip_other_spaces = re.sub("\t.*", "", strip_first_spaces)
                    ord_txt_cols[key].append(strip_other_spaces)
        except:
            pass


# Correct errors from DataDocumentation.txt
ord_txt_cols["BsmtFin Type 2"] = ord_txt_cols.pop("BsmtFinType 2")
ord_txt_cols["Heating QC"] = ord_txt_cols.pop("HeatingQC")
ord_txt_cols["Kitchen Qual"] = ord_txt_cols.pop("KitchenQual")
ord_txt_cols["Fireplace Qu"] = ord_txt_cols.pop("FireplaceQu")

pprint(ord_txt_cols)

{'Bsmt Cond': ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'NA'],
 'Bsmt Exposure': ['Gd', 'Av', 'Mn', 'No', 'NA'],
 'Bsmt Qual': ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'NA'],
 'BsmtFin Type 1': ['GLQ', 'ALQ', 'BLQ', 'Rec', 'LwQ', 'Unf', 'NA'],
 'BsmtFin Type 2': ['GLQ', 'ALQ', 'BLQ', 'Rec', 'LwQ', 'Unf', 'NA'],
 'Electrical': ['SBrkr', 'FuseA', 'FuseF', 'FuseP', 'Mix'],
 'Exter Cond': ['Ex', 'Gd', 'TA', 'Fa', 'Po'],
 'Exter Qual': ['Ex', 'Gd', 'TA', 'Fa', 'Po'],
 'Fence': ['GdPrv', 'MnPrv', 'GdWo', 'MnWw', 'NA'],
 'Fireplace Qu': ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'NA'],
 'Functional': ['Typ', 'Min1', 'Min2', 'Mod', 'Maj1', 'Maj2', 'Sev', 'Sal'],
 'Garage Cond': ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'NA'],
 'Garage Finish': ['Fin', 'RFn', 'Unf', 'NA'],
 'Garage Qual': ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'NA'],
 'Heating QC': ['Ex', 'Gd', 'TA', 'Fa', 'Po'],
 'Kitchen Qual': ['Ex', 'Gd', 'TA', 'Fa', 'Po'],
 'Land Slope': ['Gtl', 'Mod', 'Sev'],
 'Lot Shape': ['Reg', 'IR1', 'IR2', 'IR3'],
 'Paved Drive': ['Y', 'P', 'N'],
 'Po

Now, the features will be converted into a pandas categorical format.

In [7]:
for col, cats in ord_txt_cols.items():
    ord_txt_cat_type = CategoricalDtype(categories=cats, ordered=True)
    df[col] = df[col].astype(ord_txt_cat_type)

# 3. Data cleaning - part 1

## 3.1. Remove features (columns)

### 3.1.1. Uninformative features

In [8]:
# The code from DataQuest's solution
# (https://github.com/dataquestio/solutions/blob/master/Mission240Solutions.ipynb)
df = df.drop(["PID", "Order"], axis=1)

### 3.1.2. Features leaking info on target (`SalePrice`)

A copy of `Yr Sold` will be kept for use in section [2.3.1](#2.3.-Aggregating-features-(numerical-features%29)

In [9]:
# Save "Yr Sold" feature for later use
yr_sold = df["Yr Sold"]

# Drop the features
df = df.drop(["Mo Sold", "Sale Condition", "Sale Type", "Yr Sold"], axis=1)

### 3.1.3. Features with missing values
#### 3.1.3.1. Missing value rate > 5% (numerical features)

The 5% cut-off was suggested by DataQuest. I have not attempted to justify this value.

Show and remove features missing > 5% of instances (rows).

In [10]:
# Select numeric features
df_num = df.select_dtypes(include=np.number)

# missing value rate
mvr_num = df_num.isnull().sum() / df_num.shape[0]

# Show features with > 5% data missing
mvr_num_5 = mvr_num[mvr_num > 0.05]

print("Numerical features with > 5% missing values (n = {}):".format(len(mvr_num_5)))
display(mvr_num_5.sort_values())

# Remove them
df = df.drop(mvr_num_5.index, axis=1)

Numerical features with > 5% missing values (n = 2):


Garage Yr Blt    0.054266
Lot Frontage     0.167235
dtype: float64

#### 3.1.3.2. Missing value rate > 0% (categorical features)

Categorical features with any missing data will be removed. I am following DataQuest's suggestion again because imputation seems more complicated for categorical features than numerical ones.

In [11]:
# Select categorical features
df_cat = df.select_dtypes(include="category")

# missing value rate
mvr_cat = df_cat.isnull().sum() / df_cat.shape[0]

# Show features with > 5% data missing
mvr_cat_5 = mvr_cat[mvr_cat > 0]

print("Categorical features with any missing value (n = {}):".format(len(mvr_cat_5)))
display(mvr_cat_5.sort_values())

# Remove them
df = df.drop(mvr_cat_5.index, axis=1)

Categorical features with any missing value (n = 16):


Electrical        0.000341
Mas Vnr Type      0.007850
Bsmt Qual         0.027304
Bsmt Cond         0.027304
BsmtFin Type 1    0.027304
BsmtFin Type 2    0.027645
Bsmt Exposure     0.028328
Garage Type       0.053584
Garage Finish     0.054266
Garage Qual       0.054266
Garage Cond       0.054266
Fireplace Qu      0.485324
Fence             0.804778
Alley             0.932423
Misc Feature      0.963823
Pool QC           0.995563
dtype: float64

## 3.2. Impute missing values for numerical features
___
Note: In this step, imputation will be done with the whole dataset. That is, **imputation will be done *before* feature selection.**

This would be a correct practice according to [this post](https://stats.stackexchange.com/a/183963), considering that (1) the data have a relatively small number of features (n=82) and (2) missing data is assumed (not have been verified) to be [missing at random (MAR)](https://en.wikipedia.org/wiki/Missing_data#Missing_at_random).

However, I am not entirely sure. My intuition tells me that imputation should be done after each feature selection so that the replacement values can specifically reflect the selected feature set

I will come back to this if and when I can find time for it
___

Missing values will be imputed (i.e. replaced) with the **mode** of the relelvant feature.

Missing values can be treated either by (1) removing all the instances (rows) including the missing values or (2) imputing them. Here, I choose to impute them to avoid loss of data and for simplicity. Regarding simplicity, imputation will be simpler because it can be done just once to the **whole** data before feature selection. On the other hand, missing value removal should follow each feature selection because different subsets of features will have different sets of missing values.

In [12]:
# Select numeric features
df_num_new = df.select_dtypes(include=np.number)

# Show missing value rate for 
# numerical features with missing values
mvr_num_new = df_num_new.isnull().sum() / df_num_new.shape[0]
num_null = mvr_num_new[mvr_num_new > 0]

print("Numerical features for imputation:")
display(num_null)

Numerical features for imputation:


Mas Vnr Area      0.007850
BsmtFin SF 1      0.000341
BsmtFin SF 2      0.000341
Bsmt Unf SF       0.000341
Total Bsmt SF     0.000341
Bsmt Full Bath    0.000683
Bsmt Half Bath    0.000683
Garage Cars       0.000341
Garage Area       0.000341
dtype: float64

In [13]:
# Do imputation
num_null_cols = num_null.index
df[num_null_cols] = df[num_null_cols]\
                        .fillna(
                            df[num_null_cols]\
                            .mode()\
                            .to_dict(orient="records")[0] # This line is needed when using mode(), but not mean()
                        )

# 4. Feature engineering - part 2
## 4.1. Feature scaling (numerical features)

All numerical features will be [min-max scaled](https://en.wikipedia.org/wiki/Feature_scaling#Rescaling).

I will use my own function, but it can also be done with scikit-learn's preprocessing module (see [example](http://sebastianraschka.com/Articles/2014_about_feature_scaling.html#standardization-and-min-max-scaling)).

In [14]:
def min_max_scale(x, x_min, x_max):
    return (x - x_min) / (x_max - x_min)

for col in df.select_dtypes(include=np.number).columns:
    x_min = df[col].min()
    x_max = df[col].max()
    df[col] = df[col].apply(min_max_scale, args=(x_min, x_max))

## 4.2. Dummy coding (categorical features)

This creates a [sparse matrix](https://en.wikipedia.org/wiki/Sparse_matrix) from each categorical feature.

Sparse matrices as well as continous data can be used for sklearn's [ExtraTreesRegressor](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.ExtraTreesRegressor.html) (for feature selection) and [LinearRegression](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) (for model fitting and prediciton).

In [15]:
# Show numbers of categories from
# the feature with most categories.
num_cats = set()

for col in df.select_dtypes(include="category").columns:
    num_cats.add(df[col].unique().shape[0])

max(num_cats)

28

The greatest number of categories found in all categorical features is 28. I think this is not huge, so I will use [dummy coding](https://en.wikipedia.org/wiki/Dummy_variable_(statistics%29).

[Feature hashing](https://en.wikipedia.org/wiki/Feature_hashing) could be done instead if there were a lot more categories in a feature - I do not know what defines "a lot more".

In [16]:
# Incomplete code block for feature hashing
# which I hope to further work on another time
#
# df_cats = df.select_dtypes(include="category")
# h = FeatureHasher(n_features=10, input_type="string")
# pd.DataFrame(h.transform(df_cats).toarray())


cat_cols = df.select_dtypes(include="category").columns
dummy_cols = pd.get_dummies(df[cat_cols])

df.drop(cat_cols, axis=1, inplace=True)

df = pd.concat([df, dummy_cols], axis=1)

## 4.3. Aggregating features (numerical features)

The years of sale, construction and remodelling/addition are not useful by themselves. Therefore, this section will calculate the period from construction and remodelling/addition until their sales.

In [17]:
# Scale year of sale
yr_sold_min = yr_sold.min()
yr_sold_max = yr_sold.max()
yr_sold = yr_sold.apply(min_max_scale, args=(yr_sold_min, yr_sold_max))

# Calculate years
yrs = []
for col in df.select_dtypes(include=np.number).columns:
    if col == "Year Built":
        df["Years since construction"] = yr_sold - df[col]
    if col == "Year Remod/Add":
        df["Years since remod/add"] = yr_sold - df[col]

# 5. Data cleaning - part 2

## 5.1. Drop features which are highly correlated with each other

**Note: The code has been copied from https://chrisalbon.com/machine_learning/feature_selection/drop_highly_correlated_features/.**

In [18]:
## Identify Highly Correlated Features
# Create correlation matrix
corr_matrix = df.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find index of feature columns with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]



## Drop Marked Features
# Drop features
df = df.drop(df[to_drop].columns, axis=1)

print("Dropped columns:")
print(to_drop)

Dropped columns:
['Street_Pave', 'Bldg Type_2fmCon', 'Bldg Type_Duplex', 'House Style_SLvl', 'Exterior 2nd_CmentBd', 'Exterior 2nd_MetalSd', 'Exterior 2nd_PreCast', 'Exterior 2nd_VinylSd', 'Central Air_Y']


## 6. Predict and evaluate

K-fold cross validation will be carried out where K will range from 2 to and including 10.

In each fold, **feature selection** will be done based on each feature's importance which will be evaluated using **extremely randomised trees** algorithm ([ExtraTreesRegressor](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.ExtraTreesRegressor.html#sklearn.ensemble.ExtraTreesRegressor)).

Feature selection will be done 20 times per fold. First selection will include the most important feature. The second selection will be the first selection plus the next most important feature. The same will be done for up to a selection of 20 most important features.

With each set of selected features, predictions will be made using [linear regression](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) and then evaluated using [root-mean-square error](https://en.wikipedia.org/wiki/Root-mean-square_deviation) (RMSE). The resulting RMSEs (n=20) will be averaged within each fold.

Finally, the best fold will be selected and its RMSE will be reported together with the names and numbers of selected features.

**Note**: Feature selection is done ***during*** cross validation rather than beforehand. [This will prevent bias which can be created from using feature sets selected from the *whole* dataset to make prediction on *subsets* of data. By doing this, cross validation assesses the **model fitting process** rather than the model itself](https://stats.stackexchange.com/a/27751).

In [19]:
def feature_selection(selector, features, target, top_features_number):

    selector.fit(features, target)
    
    feature_cols_series = pd.Series(feature_cols, \
                                    name="Feature_cols")
    
    if type(selector) == ExtraTreesRegressor:
        feature_significance = selector.feature_importances_
        feature_significance_label = "Feature_importance_score"
        ascending = False

    elif type(selector) == RFE:
        feature_significance = selector.ranking_
        feature_significance_label = "Feature_importance_ranking"
        ascending = True
        
    
    feature_significance_series = pd.Series(feature_significance, \
                                          name=feature_significance_label)

    feature_cols_top = pd.concat([feature_cols_series, feature_significance_series], axis=1)\
            .sort_values(by=feature_significance_label, ascending=ascending)\
            .iloc[:top_features_number, 0].values
            
    return feature_cols_top

def train_and_test(fs_train, fs_test, t_train, t_test, model):
    
    # Fit model
    model.fit(fs_train, t_train)
    
    # Predict target using test dataset
    p_test = model.predict(fs_test)
    
    # Get RMSE (root-mean-square error)
    rmse = np.sqrt(mean_squared_error(t_test, p_test))
    
    return rmse


# Get features and target
target_col = "SalePrice"
feature_cols = df.columns.drop(target_col)

features = df[feature_cols]
target = df[target_col]

# Model to use for prediction
model = LinearRegression()

# Track lowest RMSE
lowest_rmse = df.max().max() - df.min().min()

# Track best method
# [fold, number of selected features, 
# names of selected features, lowest RMSE]
best_method = [None, None, None, lowest_rmse]


start = time()

# # initiate RFE feature selector
# estimator = SVR(kernel="linear")
# selector = RFE(estimator, 1, step=1)

# Initiate ExtraTreesRegressor feature selector
selector = ExtraTreesRegressor()

num_folds = range(2, 11)
rmse_all = {str(fold) + "-fold CV": [] for fold in num_folds}
feature_set_sizes = range(1, 21)

# K-fold cross validation
for fold in num_folds:
    
    rmse_split = []
    kf = KFold(n_splits=fold, shuffle=True)

    rmse_selections = {}
    feature_selections = {}
    
    kf_splits = kf.split(features)
    
    for (train_ind, test_ind), split in zip(kf_splits, range(fold)):

        rmse_selections[split] = []
        feature_selections[split] = []
        
        for fss in feature_set_sizes:

            # Get target
            t_train = target.loc[train_ind]
            t_test = target.loc[test_ind]
            
            # Get features
            f_train = features.loc[train_ind]
            f_test = features.loc[test_ind]
            
            # Select features
            fs_cols = feature_selection(selector, f_train, t_train, top_features_number=fss)
            
            fs_train = f_train[fs_cols]
            fs_test = f_test[fs_cols]
            
            # record RMSE and selected features
            rmse = train_and_test(fs_train, fs_test, t_train, t_test, model)
            rmse_selections[split].append(rmse)
            feature_selections[split].append(fs_cols)
        
    # Get mean RMSE and feature names per feature selection
    rmse_array = np.array([rmse_selections[i] for i in rmse_selections])
    feature_array = np.array([feature_selections[i] for i in feature_selections])
    for fs in feature_set_sizes:
        rmse_fs_mean = np.mean(rmse_array[:, fs - 1])
        features_fs = feature_array[:, fs - 1]
        
        if rmse_fs_mean < lowest_rmse:
            lowest_rmse = rmse_fs_mean
            best_method = [fold, fs, features_fs, lowest_rmse]
        
        rmse_all[str(fold) + "-fold CV"].append(rmse_fs_mean)
    
end = time()

# Get unique feature selections in best_method
# (source https://stackoverflow.com/a/3724558)
features_fs_all = [list(x) for x in set(tuple(x) for x in best_method[2])]

# Get selector name
selector_type_str = str((type(selector)))
selector_name = re.sub(".*\.|'.*$", "", selector_type_str)

print("Duration: " + ("{:.2f}").format(end - start) + " seconds")
print()

print("K-fold cross validation was tried with K ranging from {} to and including {}."\
     .format(min(num_folds), max(num_folds)))
print()
print("Feature selection was made using {}.".format(selector_name))
print("Different selection sizes were tried with")
print("smallest set including {} features".format(min(feature_set_sizes)))
print("and maximum one including {} features.".format(max(feature_set_sizes)))
print()

print("Best prediction was made with RMSE {} in".format(best_method[3]))
print("(1) {}-fold cross-validation".format(best_method[0]))
print("(2) with {} best features selected".format(best_method[1]))
print()
print("The best feature sets selected in each fold were")
print()
for ind, val in enumerate(features_fs_all):
    val = re.sub("\[|\]", "", str(val))
    to_print = str(val) + " and" if ind < len(features_fs_all) - 1 \
                              else str(val)
    
    print(to_print)
    print()
print()
print("(Each feature set lists features in order of importance)")
    
rmse_all_df = pd.DataFrame(data=rmse_all, index=[str(i) + " feature" for i in feature_set_sizes])
rmse_all_df

Duration: 717.79 seconds

K-fold cross validation was tried with K ranging from 2 to and including 10.

Feature selection was made using ExtraTreesRegressor.
Different selection sizes were tried with
smallest set including 1 features
and maximum one including 20 features.

Best prediction was made with RMSE 0.043827442585094425 in
(1) 10-fold cross-validation
(2) with 20 best features selected

The best feature sets selected in each fold were

'Exter Qual_TA', 'Garage Cars', 'Kitchen Qual_Ex', 'Gr Liv Area', 'Year Built', '1st Flr SF', 'Overall Qual_Very Good', 'Total Bsmt SF', 'Fireplaces', 'BsmtFin SF 1', '2nd Flr SF', 'Lot Area', 'Full Bath', 'Garage Area', 'Year Remod/Add', 'Kitchen Qual_TA', 'Overall Qual_Very Excellent', 'Bsmt Full Bath', 'Overall Qual_Excellent', 'Overall Qual_Good' and

'Exter Qual_TA', 'Kitchen Qual_Ex', 'Garage Cars', 'Gr Liv Area', 'Full Bath', 'Overall Qual_Very Good', '1st Flr SF', 'Fireplaces', 'Total Bsmt SF', 'BsmtFin SF 1', 'Lot Area', 'Overall Qual_Ex

,2-fold CV,3-fold CV,4-fold CV,5-fold CV,6-fold CV,7-fold CV,8-fold CV,9-fold CV,10-fold CV
1 feature,0.086974,0.086688,0.086877,0.086664,0.086810,0.086121,0.086653,0.085725,0.086227
2 feature,0.074260,0.075713,0.074115,0.075108,0.073900,0.075860,0.075035,0.073907,0.074021
3 feature,0.066215,0.066148,0.066049,0.066182,0.065252,0.065838,0.066010,0.065960,0.065872
4 feature,0.056702,0.062078,0.055506,0.055525,0.056286,0.056813,0.056574,0.055354,0.055957
5 feature,0.054295,0.053711,0.053614,0.053290,0.053227,0.053071,0.053636,0.052846,0.053544
6 feature,0.055581,0.053099,0.051905,0.051844,0.052700,0.052382,0.052766,0.052103,0.052657
7 feature,0.054784,0.050438,0.052060,0.050685,0.052278,0.050685,0.051524,0.051487,0.052437
8 feature,0.052701,0.050633,0.051724,0.049046,0.050816,0.051606,0.050998,0.049935,0.050522
9 feature,0.053490,0.048890,0.049217,0.050200,0.049107,0.049104,0.049494,0.049778,0.049624
10 feature,0.052833,0.048313,0.049855,0.050443,0.047987,0.049420,0.049951,0.048847,0.048285


Using linear regression, the lowest prediction error (RMSE≈0.044) was achieved when 20 most important features were used in 10-fold cross-validation.

# 7. Closing remarks

## 7.1. Limitation
### 7.1.1. Imputation done using whole dataset

In step [3.2](#3.2.-Impute-missing-values-for-numerical-features), imputation was carried out before feature selection. My intuition is that the current imputation done using whole dataset must have created a bias in each set of selected features.

### 7.1.2. Feature selection method

The output of the [previous step](#4.-Predict-and-evaluate) will be different each time it is run. This is because extra trees regressor is a method with intrinsic randomness. Intuitively, this is not good for making consistent predictions. I used it here because it was much faster than the recursive feature elimination (the only other method that I tried using).

## 7.2. Other methods tried

For feature selection, other than the extra trees regressor that I used above, I also tried using [recursive feature elimination](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html#sklearn.feature_selection.RFE). However, it took too long to run on my computer, so I discarded it.